In [ ]:
import sys
sys.path.append('../')

In [ ]:
import dill
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm_notebook
from plotly.offline import init_notebook_mode, iplot
from plotly import graph_objs as go

#from src_nlp_pkg.text_preprocessing.text_preprocessor import TextPreprocessor as TP
from utilities_nlp_pkg.one_hot_encoder import OneHotEncoder
from utilities_nlp_pkg.word2vec_preprocessor import Word2VecPreprocessor as TP

init_notebook_mode()

In [ ]:
embedding_dim = 3
iterations = 1500
language = 'ENG'

corpus = {
    'ENG': [
        'man is strong', 'woman is pretty', 'boy is a young man', 'king is a strong man', 'girl is a young woman', 
        'queen is a wise woman', 'prince is a young king', 'princess is a young queen', 
        'a boy who is a prince will be king', 'a girl who is a princess will be queen'
    ],
    'RUS': [
        'такой система найти широкий применение образование', 
        'такой информационный система главный образ решение практический учебный задача возникать структурированный формализуемый предметный область образование', 
        'основа концепция интеллектуальность умение работать формализованный знание человек', 
        'проблема поиск прогрессивный математический метод анализ проектированиу образовательный сложный система актуальный',
        'наблюдать повышенный интерес разработка область экспертный система учебный назначение широкий слой специалист разный профиль число преподаватель', 
        'отмечать интерес распределенный интеллектуальный система сетевой обучение четкий понимание организовать образовательный процесс получение желаемый результат', 
        'построение педагогический образовательный модель сетевой информационный образовательный среда',
        'теоретический педагогический аспект эффективность применение сетевой обучение требовать новый способ прием организация совместный учебный деятельность сетевой взаимодействие учащийся преподаватель', 
        'основной свойство интерактивный обучение открытый форма информационный обмен внешний среда число доступ знание профессиональный педагог эксперт который иметь личный работа сетевой технология прогнозировать развитие технология',
        'знание такой предметный область нуждаться экспертный система', 
        'возникать проблема связанный многий исследователь область технология сетевой обучение переносить традиционный педагогический методика сетевой обучение учет специфика', 
        'традиционный лекция учебник сетевой обучение специфика сетевой обучение подразумевать организованный направленный открытый доступ динамичный система актуализуемый информация доступный онлайн консультация новый методический подход организация совместный проектный деятельность другой',
        'технология сетевой обучение многий случай предполагать возможность выбор учащийся собственный расписание темп ритм обучение', 
        'возрастать нагрузка сетевой преподаватель модератор тьютор', 
        'индивидуализация снижение данный рутинный нагрузка использование подсистема',     
        'подсистема обеспечить развитый автоматизированный система интеллектуальный подсказка помощь онлайн консультация сетевой обучение использование разный образовательный метод обучение лекция практикум проектный деятельность конференция деловой интерактивный тренинг',
        'уникальный вопрос адресоваться преподаватель эксперт',
        'анализ применение информационный система автоматизация учебный процесс образовательный учреждение выявить противоречие нарушающий основной тенденция современный образование дифференциация интеграция показать отсутствие целенаправленный использование управление учебный процесс соответствие требуемый принцип дидактический система', 
        'настоящий время оставаться актуальный выявление основной противоречие управление процесс обучение определение средство решение основа использование современный информационный технология первый очередь экспертный система',
        'устранение указанный противоречие служить введение учебный процесс подсистема осуществляющий процесс обучение адаптивный блок интеллектуальный поддержка процесс обучение интеллектуализация информационный система', 
        'путь создание интеллектуальный подсистема модуль позволяющий проводить обучение аттестация анализировать эффективность обучение основа тест созданный эксперт индивидуализировать процесс обучение'
    ]
}

corpus = TP.remove_stop_words(corpus[language], language)
words = TP.get_unique_words(corpus)
ohe = OneHotEncoder(words, False)

In [ ]:
df = TP.get_words_and_context(corpus)
df.head()

In [ ]:
X, Y = ohe.split_to_x_y(df)

word_ph = tf.placeholder(tf.float32, shape=(None, ohe.dim))
context_ph = tf.placeholder(tf.float32, shape=(None, ohe.dim))

weights_hidden = tf.Variable(tf.random_normal([ohe.dim, embedding_dim]))
bias_hidden = tf.Variable(tf.random_normal([1]))
hidden_layer = tf.add(tf.matmul(word_ph, weights_hidden), bias_hidden)

weights_pred = tf.Variable(tf.random_normal([embedding_dim, ohe.dim]))
bias_pred = tf.Variable(tf.random_normal([1]))
prediction = tf.nn.softmax(tf.add(tf.matmul(hidden_layer, weights_pred), bias_pred))

loss = tf.reduce_mean(-tf.reduce_sum(context_ph * tf.log(prediction), axis=[1]))

train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

PBAR = tqdm_notebook(total = iterations, desc = 'Word2Vec')
for i in range(iterations):
    sess.run(train_op, feed_dict={word_ph: X, context_ph: Y})
    _loss = sess.run(loss, feed_dict={word_ph: X, context_ph: Y})
    PBAR.update()
    PBAR.write('Current loss is {:10.4f}%'.format(_loss), end = '\r')

In [ ]:
vectors = sess.run(weights_hidden + bias_hidden)
#dill.dump(vectors, open('../dumps/vectors.dill', 'wb'))
#vectors = dill.load(open('../dumps/vectors.dill', 'rb'))
cols = ['x1', 'x2', 'x3'][:embedding_dim]
w2v_df = pd.DataFrame(vectors, columns = cols)
w2v_df['word'] = words
w2v_df = w2v_df[['word'] + cols]
w2v_df.head()

In [ ]:
if embedding_dim == 2:
    trace = go.Scatter(
        x = w2v_df.x1,
        y = w2v_df.x2,
        text = w2v_df.word,
        mode = 'markers+text',
        marker = dict(
            size = 1
        ),
        textfont = dict(
            size = 18
        )
    )
else:
    trace = go.Scatter3d(
        x = w2v_df.x1,
        y = w2v_df.x2,
        z = w2v_df.x3,
        text = w2v_df.word,
        mode = 'markers+text',
        marker = dict(
            size = 1
        ),
        textfont = dict(
            size = 18
        )
    )

layout = go.Layout(
    height=768
)

iplot({'data': [trace], 'layout': layout}, show_link=False)

In [ ]:
if language != 'ENG':
    pass
else:
    w2v = w2v_df.copy()
    w2v.index = w2v.word
    w2v.drop('word', 1, inplace=True)
    analogy = w2v.loc['king', 'x1':'x2'] - w2v.loc['man', 'x1':'x2'] + w2v.loc['woman', 'x1':'x2']
    w2v['similarity'] = (w2v['x1'] - analogy.x1) ** 2 + (w2v['x2'] - analogy.x2) ** 2
    
    w2v.sort_values('similarity').head(3)